## Obtener el error de picota, leyendo el pdf de resulatdos

In [1]:
import fitz  # PyMuPDF

doc = fitz.open("../../data/report.pdf")

for page_num, page in enumerate(doc, start=1):
    text = page.get_text()
    print(f"Página {page_num}:")
    print(text)
    print("-" * 80)


Página 1:
Inferential DataSheet
picota
Model Identifier: b9ed413b-68c7-487f-bea8-81887a538521
https://quassar.io/commits/b9ed413b-68c7-487f-bea8-81887a538521
Title: main_48_0_1789027358   Release Date: 2025-07-01
Digital Subject: hogar09
Estimation: 
gastoMonetario:productosAlimenticios11
Inference Error: ±2,06 %
Top Contributing Inputs
Input Variable
Weight (%)
ingresosNetos
18,00%
tasaAhorro
16,00%
comidasTotales
14,00%
ipc
8,00%
Estimation: 
gastoMonetario:bebidasNoAlcoholicas12
Inference Error: ±2,27 %
Top Contributing Inputs
Input Variable
Weight (%)
ingresosNetos
16,00%
tasaAhorro
15,00%
comidasTotales
10,00%
ipc
9,00%
Estimation: 
gastoMonetario:bebidasAlcoholicas21
Inference Error: ±,91 %
Top Contributing Inputs
Estimation: gastoMonetario:tabaco22
Inference Error: ±2,64 %
Top Contributing Inputs
Input Variable
Weight (%)
tasaAhorro
13,00%

--------------------------------------------------------------------------------
Página 2:
Input Variable
Weight (%)
ingresosNetos
22,00%
ta

In [2]:
import fitz
import re
import pandas as pd

# Ruta a tu PDF
ruta_pdf = "../../data/report.pdf"

# Abrir documento
doc = fitz.open(ruta_pdf)

data = []
subject_actual = None

# Recorrer páginas
for page_num, page in enumerate(doc, start=1):
    text = page.get_text()

    # Buscar Digital Subject en la página
    m_subject = re.search(r"Digital Subject:\s*(\w+)", text)
    if m_subject:
        subject_actual = m_subject.group(1)

    # Buscar bloques de Estimation
    bloques = re.findall(
        r"Estimation:\s*(gastoMonetario:[^\n]*)\nInference Error:\s*±([\d,\.]+)\s*%",
        text
    )

    for var, err in bloques:
        # Quitar prefijo "gastoMonetario:"
        var_clean = var.replace("gastoMonetario:", "")
        # Normalizar decimal (puede venir coma o punto)
        err_val = float(err.replace(",", "."))
        data.append({
            "subject": subject_actual,
            "gastoMonetario": var_clean,
            "inference_error_pct": err_val
        })

# Pasar a DataFrame
df_errors = pd.DataFrame(data)

print(df_errors)


     subject           gastoMonetario  inference_error_pct
0    hogar09  productosAlimenticios11                 2.06
1    hogar09   bebidasNoAlcoholicas12                 2.27
2    hogar09     bebidasAlcoholicas21                 0.91
3    hogar09                 tabaco22                 2.64
4    hogar09      articulosDeVestir31                 1.83
..       ...                      ...                  ...
831  hogar14      proteccionSocial124                 0.68
832  hogar14               seguros125                 1.79
833  hogar14  serviciosFinancieros126                 2.05
834  hogar14        otrosServicios127                 0.24
835  hogar14               remesas128                 0.58

[836 rows x 3 columns]


In [3]:
df_errors

,subject,gastoMonetario,inference_error_pct
0,hogar09,productosAlimenticios11,2.06
1,hogar09,bebidasNoAlcoholicas12,2.27
2,hogar09,bebidasAlcoholicas21,0.91
3,hogar09,tabaco22,2.64
4,hogar09,articulosDeVestir31,1.83
...,...,...,...
831,hogar14,proteccionSocial124,0.68
832,hogar14,seguros125,1.79
833,hogar14,serviciosFinancieros126,2.05
834,hogar14,otrosServicios127,0.24


In [4]:
pivot_df = df_errors.pivot_table(
    index="subject",
    columns="gastoMonetario",
    values="inference_error_pct"
)

# Quitamos nombre de las columnas (solo estético)
pivot_df.columns.name = None

# EXTRA: Ordenar columnas por código numérico
# Extraemos el número final de cada columna
col_codes = pivot_df.columns.to_series().apply(
    lambda x: int(''.join(filter(str.isdigit, x[-4:])))
)

# Ordenamos las columnas según esos códigos
ordered_cols = col_codes.sort_values().index.tolist()

# Reordenamos el DataFrame
pivot_df = pivot_df[ordered_cols]


In [5]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pivot_df

,productosAlimenticios11,bebidasNoAlcoholicas12,bebidasAlcoholicas21,tabaco22,articulosDeVestir31,calzado32,alquileresRealesDeLaVivienda41,mantenimientoDeLaVivienda43,suministroDeAgua44,electricidadGasOtrosCombustibles45,mueblesRevestimientos51,textilesParaElHogar52,grandesElectrodomesticos53,utensiliosDelHogar54,herramientasCasaJardin55,bienesServiciosParaElHogar56,productosFarmaceuticos61,serviciosMedicosAmbulatorios62,serviciosHospitalarios63,compraDeVehiculos71,usoDeVehiculosPersonales72,serviciosDeTransporte73,serviciosPostales81,equiposTelefonoFax82,serviciosTelefonoFax83,audiovisualesTecnologia91,bienesDuraderosDeOcio92,ocioJardineriaYMascotas93,serviciosRecreativosYCulturales94,prensaYPapeleria95,paquetesTuristicos96,educacionInfantilYPrimaria101,educacionSecundariaYPostsecundaria102,educacionSuperior103,educacionNoFormal104,restauracion111,alojamiento112,cuidadosPersonales121,efectosPersonales123,proteccionSocial124,seguros125,serviciosFinancieros126,otrosServicios127,remesas128
subject,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hogar01,2.76,2.42,0.47,1.44,1.36,2.21,0.86,0.65,0.44,2.72,0.52,0.61,1.42,0.56,0.37,0.62,1.24,0.47,0.12,1.55,2.12,1.46,0.10,1.19,2.56,1.54,0.10,0.67,0.32,0.75,0.89,0.48,0.50,0.70,0.35,1.96,0.98,1.26,0.18,0.37,0.80,0.63,0.06,0.15
hogar02,4.29,2.43,1.50,3.81,1.16,2.49,2.15,0.87,2.18,3.32,1.22,0.94,2.62,0.48,0.39,1.37,1.36,1.36,0.06,1.92,2.68,2.18,0.14,2.42,5.03,2.37,0.23,1.05,1.61,1.15,1.86,0.67,0.47,0.66,0.68,2.92,2.38,2.19,0.48,0.92,1.78,1.80,0.15,0.74
hogar03,3.37,2.85,1.80,2.59,2.49,3.67,2.07,0.70,0.64,1.51,1.30,0.54,1.58,0.29,0.61,1.65,1.85,0.78,0.10,2.28,2.70,1.47,0.16,2.38,3.48,1.20,0.30,1.01,0.97,1.42,2.01,0.39,0.56,0.61,0.21,1.89,0.97,1.52,0.51,0.90,0.84,2.64,0.07,0.76
hogar04,3.51,3.10,1.64,4.42,1.53,3.40,1.91,0.76,1.42,3.82,0.54,0.76,2.07,0.63,0.67,1.82,1.72,1.10,0.10,2.95,2.24,3.18,0.52,2.10,4.52,1.34,0.08,0.74,1.23,0.68,1.25,1.16,0.72,0.56,0.98,2.11,1.35,3.68,0.26,0.96,2.85,2.03,0.27,0.53
hogar05,4.68,4.50,2.11,1.95,1.47,2.88,1.98,1.19,0.72,4.14,1.37,0.67,2.35,0.55,0.73,0.86,1.32,0.52,0.15,1.98,2.36,1.69,0.14,1.34,2.05,0.87,0.08,0.96,1.94,1.80,1.20,0.96,0.89,0.82,0.63,2.23,1.46,3.25,0.48,0.62,1.28,2.35,0.18,0.42
hogar06,3.35,2.51,1.95,3.76,1.99,2.99,1.51,0.82,1.45,4.79,1.54,1.00,1.80,0.40,0.98,2.48,1.25,1.20,0.16,2.48,2.52,3.19,0.23,2.44,3.31,0.81,0.14,1.63,1.79,2.28,2.00,0.84,0.89,0.49,1.55,3.65,2.07,1.91,0.72,0.45,0.97,0.94,0.26,0.56
hogar07,2.51,2.21,0.32,2.54,1.31,1.62,1.33,0.64,1.45,1.68,1.08,1.01,1.56,0.27,0.37,0.59,1.83,0.87,0.08,1.58,1.52,1.06,0.22,1.97,3.18,2.26,0.07,0.87,0.57,0.59,2.09,0.70,0.43,0.50,0.36,2.93,1.43,1.26,0.39,0.61,0.84,1.72,0.14,0.59
hogar08,2.80,4.04,1.06,2.52,0.96,3.65,2.00,0.77,1.04,3.43,0.87,1.03,2.02,0.40,0.76,1.67,1.75,0.56,0.09,1.67,2.17,2.12,0.13,2.19,4.08,1.12,0.18,1.58,1.08,0.93,2.40,0.48,0.48,0.46,0.88,2.30,1.28,1.23,0.18,0.59,1.63,1.40,0.27,0.34
hogar09,2.06,2.27,0.91,2.64,1.83,3.28,1.62,0.81,1.03,1.96,0.64,0.47,2.51,0.42,0.44,1.07,1.00,0.66,0.15,2.25,2.54,2.00,0.10,2.08,3.18,0.80,0.10,1.02,0.46,0.54,0.63,1.18,0.49,0.76,0.32,2.34,1.25,1.67,0.32,0.27,0.80,2.04,0.13,0.20


In [9]:
# Guardar el DataFrame final en un archivo TSV
pivot_df.to_csv("../../datamarts/inference_errors.tsv", sep="\t", index=True)

#  Obtener error

In [15]:
import os
import json
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

In [16]:
import os
import pandas as pd

def leer_tsv_ccaa(ccaa, ruta_tsvs):
    """
    Lee el archivo hogar{ccaa}.tsv dentro del directorio indicado y devuelve
    solo las filas correspondientes al año 2023, filtrando por timestamp.

    Parameters
    ----------
    ccaa : str o int
        Código de dos dígitos de la CCAA (e.g. "01" o 1).
    ruta_tsvs : str
        Ruta al directorio donde están los archivos .tsv

    Returns
    -------
    pd.DataFrame
        DataFrame con el contenido del archivo leído, filtrado a año 2023.
    """
    # Formatear el número a dos dígitos si viene como entero
    ccaa_str = f"{int(ccaa):02d}"
    
    nombre_archivo = f"hogar{ccaa_str}.tsv"
    ruta_archivo = os.path.join(ruta_tsvs, nombre_archivo)
    
    if os.path.exists(ruta_archivo):
        print(f"✅ Leyendo archivo: {nombre_archivo}")
        df = pd.read_csv(ruta_archivo, sep='\t')

        # Filtrar solo filas cuyo timestamp empieza por "2023"
        if "timestamp" in df.columns:
            mask = df["timestamp"].astype(str).str.startswith("2023")
            df = df[mask]
        else:
            print("⚠️ Columna 'timestamp' no encontrada en el archivo. No se filtra por año.")
        
        return df
    else:
        print(f"⚠️ Archivo no encontrado: {nombre_archivo}")
        return pd.DataFrame()  # Devuelve DF vacío si no existe


In [17]:
# Por ejemplo:
ruta_tsvs = "../../datamarts/picotaData"
ccaa = "09"

df = leer_tsv_ccaa(ccaa, ruta_tsvs)

df


✅ Leyendo archivo: hogar09.tsv


,timestamp,capitalProvincia,tamanoMunicipio,densidad,superficie,tipoCasa,aguaCaliente,calefaccion,zonaResidencial,regimenTenencia,comidasTotales,miembros:estudiantes,miembros:noEstudiantes,miembros:ocupados,miembros:noOcupados,miembros:activos,miembros:noActivos,miembros:ancianos:masculinos,miembros:ancianos:femeninos,miembros:adultos:masculinos,miembros:adultos:femeninos,miembros:ninos:femeninos,miembros:ninos:masculinos,miembros:conIngresos:masculinos,miembros:conIngresos:femeninos,miembros:sinIngresos:masculinos,miembros:sinIngresos:femeninos,edadSp,espanolSp,educacionSuperiorSp,fuentePrincipalIngresos,numeroViviendasAdicionales,ingresosNetos,gastoNoMonetario,tasaAhorro,temperaturaMedia,tasaParo,inflacion,España.tipoInteres,España.tasaCambioEurUsd,ipc,gastoMonetario:productosAlimenticios11,gastoMonetario:bebidasNoAlcoholicas12,gastoMonetario:bebidasAlcoholicas21,gastoMonetario:tabaco22,gastoMonetario:articulosDeVestir31,gastoMonetario:calzado32,gastoMonetario:alquileresRealesDeLaVivienda41,gastoMonetario:mantenimientoDeLaVivienda43,gastoMonetario:suministroDeAgua44,gastoMonetario:electricidadGasOtrosCombustibles45,gastoMonetario:mueblesRevestimientos51,gastoMonetario:textilesParaElHogar52,gastoMonetario:grandesElectrodomesticos53,gastoMonetario:utensiliosDelHogar54,gastoMonetario:herramientasCasaJardin55,gastoMonetario:bienesServiciosParaElHogar56,gastoMonetario:productosFarmaceuticos61,gastoMonetario:serviciosMedicosAmbulatorios62,gastoMonetario:serviciosHospitalarios63,gastoMonetario:compraDeVehiculos71,gastoMonetario:usoDeVehiculosPersonales72,gastoMonetario:serviciosDeTransporte73,gastoMonetario:serviciosPostales81,gastoMonetario:equiposTelefonoFax82,gastoMonetario:serviciosTelefonoFax83,gastoMonetario:audiovisualesTecnologia91,gastoMonetario:bienesDuraderosDeOcio92,gastoMonetario:ocioJardineriaYMascotas93,gastoMonetario:serviciosRecreativosYCulturales94,gastoMonetario:prensaYPapeleria95,gastoMonetario:paquetesTuristicos96,gastoMonetario:educacionInfantilYPrimaria101,gastoMonetario:educacionSecundariaYPostsecundaria102,gastoMonetario:educacionSuperior103,gastoMonetario:educacionNoFormal104,gastoMonetario:restauracion111,gastoMonetario:alojamiento112,gastoMonetario:cuidadosPersonales121,gastoMonetario:efectosPersonales123,gastoMonetario:proteccionSocial124,gastoMonetario:seguros125,gastoMonetario:serviciosFinancieros126,gastoMonetario:otrosServicios127,gastoMonetario:remesas128
32037,2023-01-01T00:00:00Z,False,1,2,90.0,2,True,True,1,5,42.0,0,3,1.0,2,1.0,2,0,0,0,1,0,2,0,1,2,0,41,True,True,asalariado,0.0,10308.0,5862.70,-0.010670,17.62,9.3,3.2,4.5,1.08,111.656167,3978.48,135.11,246.41,0.057772,0.00,608.33,0.0,0.00,273.04,813.48,0.007072,0.00,0.000000,0.00,0.00,105.76,0.00,678.780000,0.0,0.000000,0.00,600.00,0.0,0.0,514.92,0.008536,0.0,0.046867,0.115544,0.00,0.0,850.0,0.0,0.0,0.0,581.41,0.00,450.17,0.000000,0.00,0.0,582.10,0.0,0.00
32038,2023-01-01T01:00:00Z,False,3,2,90.0,2,True,True,5,4,98.0,0,7,4.0,3,5.0,2,0,0,1,4,1,1,1,3,1,2,40,True,False,asalariado,0.0,49992.0,0.00,0.732668,17.62,9.3,3.2,4.5,1.08,111.656167,0.00,0.00,0.00,0.057772,0.00,0.00,6000.0,0.00,600.00,1830.00,0.007072,0.00,0.000000,0.00,0.00,0.00,0.00,0.007072,0.0,0.000000,0.00,1303.51,0.0,0.0,1476.00,1082.430000,0.0,0.046867,0.115544,0.00,0.0,160.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.00,0.0,912.50
32039,2023-01-01T02:00:00Z,False,1,2,90.0,2,True,True,1,6,20.0,0,1,0.0,1,1.0,0,0,0,0,1,0,0,0,1,0,0,62,True,False,pension,0.0,14652.0,3435.81,-0.124336,17.62,9.3,3.2,4.5,1.08,111.656167,3875.50,1005.00,0.00,503.890000,0.00,0.00,0.0,0.00,1032.18,2773.92,0.000000,0.00,0.007072,0.00,0.00,622.38,3089.11,0.000000,0.0,0.016013,55.00,0.00,0.0,0.0,527.88,0.000000,0.0,0.033741,0.000000,0.00,780.0,0.0,0.0,0.0,0.0,570.33,0.00,1145.28,0.023434,0.00,0.0,493.29,0.0,0.00
32040,2023-01-01T03:00:00Z,True,5,3,65.0,2,True,True,5,5,11.0,0,1,1.0,0,1.0,0,0,0,0,1,0,0,0,1,0,0,46,True,True,asalariado,0.0,26520.0,6681.85,0.665614,17.62,9.3,3.2,4.5,1.08,111.656167,1605.79,0.00,0.00,0

In [18]:
API_ID = "b9ed413b-68c7-487f-bea8-81887a538521"

In [19]:
bool_cols = [
    "capitalProvincia",
    "aguaCaliente",
    "calefaccion",
    "espanolSp",
    "educacionSuperiorSp"
]

In [ ]:
# convertir fila a payload
def convertir_fila_a_payload

In [37]:
# Leer datos
df = leer_tsv_ccaa(ccaa, ruta_tsvs)

if df.empty:
    print("🚫 No hay datos.")
else:
    n_filas = min(20, len(df))
    lista_dif_abs = []
    lista_resultados = []

    for i in tqdm(range(n_filas), desc="Procesando hogares"):
        fila = df.iloc[i]
        
        # Convertir fila a payload
        payload = convertir_fila_a_payload(fila, bool_cols)
        
        # Llamada a la API
        pred_json = llamar_api(payload, ccaa, API_ID)
        if pred_json is None:
            continue
        
        # Convertir respuesta a DataFrame
        pred_df = pd.DataFrame(pred_json)
        if pred_df.empty or "variable" not in pred_df.columns:
            continue

        # Pivotar predicciones
        pred_pivot = pred_df.pivot_table(
            index=None,
            columns="variable",
            values="value"
        )
        
        cols_gasto = [
            col for col in pred_pivot.columns
            if col.startswith("gastoMonetario:")
        ]
        
        if len(cols_gasto) == 0:
            continue

        # Valores reales y predichos
        original_vals = fila[cols_gasto]
        pred_vals = pred_pivot[cols_gasto].iloc[0]
        
        # Calcular margen de error sobre el valor REAL
        min_vals = []
        max_vals = []

        for var in cols_gasto:
            # Quitar prefijo para buscar en pivot_df
            nombre_en_pivot_df = var.replace("gastoMonetario:", "")
            
            # Sacar nombre del hogar (e.g. hogar09)
            hogar_str = f"hogar{ccaa}"
            
            # Buscar error en pivot_df
            if (hogar_str in pivot_df.index) and (nombre_en_pivot_df in pivot_df.columns):
                error_pct = pivot_df.loc[hogar_str, nombre_en_pivot_df] / 100
            else:
                error_pct = 0.0

            real_value = original_vals[var]
            
            if real_value == 0:
                val_min = 0
                val_max = 0
            else:
                val_min = real_value * (1 - error_pct)
                val_max = real_value * (1 + error_pct)
                val_min = max(val_min, 0)

            min_vals.append(val_min)
            max_vals.append(val_max)
        
        # Calcular diferencia absoluta
        diff_abs = np.abs(pred_vals - original_vals)
        
        # Guardar resultados para este hogar
        df_resultado = pd.DataFrame({
            "variable": cols_gasto,
            "real": original_vals.values,
            "predicho": pred_vals.values,
            "valor_minimo_esperado": min_vals,
            "valor_maximo_esperado": max_vals,
            "error_abs": diff_abs.values,
            "hogar": i
        })
        lista_resultados.append(df_resultado)
        
        # Guardar diferencias absolutas (para MAE)
        lista_dif_abs.append(diff_abs)
    
    # -------------------------------
    # Mostrar resultados hogar a hogar
    # -------------------------------
    if lista_resultados:
        df_todos = pd.concat(lista_resultados, ignore_index=True)

        for hogar_id in sorted(df_todos["hogar"].unique()):
            df_hogar = df_todos[df_todos["hogar"] == hogar_id]
            
            print(f"\n🏠 Hogar #{hogar_id}")
            print(
                df_hogar[
                    ["variable", "real", "predicho", "valor_minimo_esperado", "valor_maximo_esperado"]
                ].to_string(index=False)
            )
        
        # -------------------------------
        # Calcular errores medios
        # -------------------------------
        df_diffs = pd.DataFrame(lista_dif_abs)
        
        mae_por_categoria = df_diffs.mean().sort_values(ascending=False)
        print("\n📊 Error Medio Absoluto por categoría (en euros):")
        print(mae_por_categoria)
        
        mae_total = df_diffs.stack().mean()
        print(f"\n✅ Error Medio Absoluto TOTAL sobre {n_filas} hogares: {mae_total:.2f} €")
    else:
        print("🚫 No se obtuvieron predicciones para comparar.")


✅ Leyendo archivo: hogar09.tsv


Procesando hogares: 100%|██████████| 20/20 [00:53<00:00,  2.67s/it]


🏠 Hogar #0
                                            variable      real    predicho  valor_minimo_esperado  valor_maximo_esperado
                       gastoMonetario:alojamiento112       0.0  141.126488               0.000000               0.000000
       gastoMonetario:alquileresRealesDeLaVivienda41       0.0   75.297260               0.000000               0.000000
                  gastoMonetario:articulosDeVestir31       0.0  423.563413               0.000000               0.000000
            gastoMonetario:audiovisualesTecnologia91  0.008536  102.254272               0.008468               0.008605
                 gastoMonetario:bebidasAlcoholicas21    246.41   87.921162             244.167669             248.652331
               gastoMonetario:bebidasNoAlcoholicas12    135.11  211.441761             132.043003             138.176997
              gastoMonetario:bienesDuraderosDeOcio92       0.0   17.435541               0.000000               0.000000
         gastoMoneta

In [39]:
# df_hogar sin error_abs ni hogar
df_hogar = df_hogar.drop(columns=["error_abs", "hogar"])
df_hogar

,variable,real,predicho,valor_minimo_esperado,valor_maximo_esperado
836,gastoMonetario:alojamiento112,1094.81,857.955306,1081.124875,1108.495125
837,gastoMonetario:alquileresRealesDeLaVivienda41,1526.22,1122.650948,1501.495236,1550.944764
838,gastoMonetario:articulosDeVestir31,4623.35,1406.098382,4538.742695,4707.957305
839,gastoMonetario:audiovisualesTecnologia91,250.0,322.503955,248.000000,252.000000
840,gastoMonetario:bebidasAlcoholicas21,0.0,316.178568,0.000000,0.000000
841,gastoMonetario:bebidasNoAlcoholicas12,723.71,326.712167,707.281783,740.138217
842,gastoMonetario:bienesDuraderosDeOcio92,0.0,15.567454,0.000000,0.000000
843,gastoMonetario:bienesServiciosParaElHogar56,2699.59,448.187396,2670.704387,2728.475613
844,gastoMonetario:calzado32,486.67,314.910680,470.707224,502.632776
845,gastoMonetario:compraDeVehiculos71,0.0,1073.558070,0.000000,0.000000


In [42]:
df_hogar.to_csv("error.tsv", sep="\t", index=False)

In [50]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Leer datos
df = leer_tsv_ccaa(ccaa, ruta_tsvs)

if df.empty:
    print("🚫 No hay datos.")
else:
    n_filas = min(20, len(df))
    lista_resultados = []

    for i in tqdm(range(n_filas), desc="Procesando hogares"):
        fila = df.iloc[i]

        # Convertir fila a payload
        payload = convertir_fila_a_payload(fila, bool_cols)

        # Llamada a la API
        pred_json = llamar_api(payload, ccaa, API_ID)
        if pred_json is None:
            continue

        # Convertir respuesta a DataFrame
        pred_df = pd.DataFrame(pred_json)
        if pred_df.empty or "variable" not in pred_df.columns:
            continue

        # Pivotar predicciones
        pred_pivot = pred_df.pivot_table(
            index=None,
            columns="variable",
            values="value"
        )

        cols_gasto = [
            col for col in pred_pivot.columns
            if col.startswith("gastoMonetario:")
        ]

        if len(cols_gasto) == 0:
            continue

        # Valores reales y predichos
        original_vals = fila[cols_gasto]
        pred_vals = pred_pivot[cols_gasto].iloc[0]

        # Calcular diferencias absolutas
        diff_abs = np.abs(pred_vals - original_vals)

        # Guardar resultados para este hogar
        df_resultado = pd.DataFrame({
            "variable": cols_gasto,
            "real": original_vals.values,
            "predicho": pred_vals.values,
            "error_abs": diff_abs.values,
            "hogar": i
        })

        lista_resultados.append(df_resultado)

    # -------------------------------------------------------
    # Calcular estadísticas si hay resultados
    # -------------------------------------------------------
    if lista_resultados:
        df_todos = pd.concat(lista_resultados, ignore_index=True)

        # -------------------------------
        # Calcular gasto real medio por categoría
        # -------------------------------
        df_nozero = df_todos[df_todos["real"] > 1e-8]
        
        gasto_medio_cat = (
            df_nozero
            .groupby("variable")
            .agg(media_real=("real", "mean"))
        )

        # Unir gasto medio a cada fila
        df_todos = df_todos.merge(
            gasto_medio_cat,
            on="variable",
            how="left"
        )

        # Calcular error relativo (%)
        df_todos["error_relativo_pct"] = np.where(
            df_todos["media_real"].notna() & (df_todos["media_real"] > 1e-8),
            (df_todos["error_abs"] / df_todos["media_real"]) * 100,
            np.nan
        )

        # Separar casos:
        df_nozero = df_todos[df_todos["real"] > 1e-8]
        df_zero = df_todos[df_todos["real"] <= 1e-8]

        # Estadísticas de errores relativos (para reales ≠ 0)
        if not df_nozero.empty:
            df_relativos = (
                df_nozero
                .groupby("variable")
                .agg(
                    mean_relative_error_pct = ("error_relativo_pct", "mean"),
                    std_relative_error_pct = ("error_relativo_pct", "std"),
                    num_hogares_nonzero = ("hogar", "count"),
                    mae_eur = ("error_abs", "mean")
                )
                .sort_values("mean_relative_error_pct", ascending=False)
            )
        else:
            df_relativos = pd.DataFrame(columns=[
                "mean_relative_error_pct",
                "std_relative_error_pct",
                "num_hogares_nonzero",
                "mae_eur"
            ])

        # Estadísticas de errores absolutos (para reales = 0)
        if not df_zero.empty:
            df_abs_zero = (
                df_zero
                .groupby("variable")
                .agg(
                    mean_absolute_error_eur = ("error_abs", "mean"),
                    std_absolute_error_eur = ("error_abs", "std"),
                    num_hogares_zero = ("hogar", "count")
                )
                .sort_values("mean_absolute_error_eur", ascending=False)
            )
        else:
            df_abs_zero = pd.DataFrame(columns=[
                "mean_absolute_error_eur",
                "std_absolute_error_eur",
                "num_hogares_zero"
            ])

        # Mostrar errores relativos
        if not df_relativos.empty:
            print("\n📈 Estadísticas de error relativo (%), para categorías con gasto real distinto de cero:")
            print(
                df_relativos[
                    ["mae_eur", "mean_relative_error_pct", "std_relative_error_pct", "num_hogares_nonzero"]
                ].to_string(float_format=lambda x: f"{x:.2f}")
            )
        else:
            print("\n⚠️ No hay datos con gasto real distinto de cero para calcular error relativo.")

        # Mostrar errores absolutos
        if not df_abs_zero.empty:
            print("\n📈 Estadísticas de error absoluto (€), para categorías con gasto real igual a cero:")
            print(
                df_abs_zero[
                    ["mean_absolute_error_eur", "std_absolute_error_eur", "num_hogares_zero"]
                ].to_string(float_format=lambda x: f"{x:.2f}")
            )
        else:
            print("\n⚠️ No hay datos con gasto real igual a cero para calcular error absoluto.")

    else:
        print("🚫 No se obtuvieron predicciones para comparar.")


✅ Leyendo archivo: hogar09.tsv


Procesando hogares: 100%|██████████| 20/20 [00:53<00:00,  2.70s/it]


📈 Estadísticas de error relativo (%), para categorías con gasto real distinto de cero:
                                                      mae_eur  mean_relative_error_pct  std_relative_error_pct  num_hogares_nonzero
variable                                                                                                                           
gastoMonetario:efectosPersonales123                     19.21                 98394.65                55201.92                   12
gastoMonetario:compraDeVehiculos71                    2960.07                   549.71                  776.50                   13
gastoMonetario:ocioJardineriaYMascotas93               542.61                   234.59                  172.36                   20
gastoMonetario:herramientasCasaJardin55                305.30                   185.27                  177.19                    2
gastoMonetario:mantenimientoDeLaVivienda43             538.01                   136.41                  130.47          

In [ ]:
import re

# Función para extraer el número al final del nombre de variable
def extract_code(variable_name):
    match = re.search(r"(\d+)$", variable_name)
    if match:
        return int(match.group(1))
    else:
        return float('inf')  # Para variables sin código numérico

# Crear columna auxiliar con el código
df_relativos = df_relativos.copy()
df_relativos["codigo_categoria"] = df_relativos.index.map(extract_code)

# Ordenar por el código
df_relativos_sorted = df_relativos.sort_values("codigo_categoria")

# Opcional: quitar la columna auxiliar si no la quieres
df_relativos_sorted = df_relativos_sorted.drop(columns=["codigo_categoria"])

# Mostrar el resultado ordenado
print(
    df_relativos_sorted[
        ["mae_eur", "mean_relative_error_pct", "std_relative_error_pct", "num_hogares_nonzero"]
    ].to_string(float_format=lambda x: f"{x:.2f}")
)


                                                      mae_eur  mean_relative_error_pct  std_relative_error_pct  num_hogares_nonzero
variable                                                                                                                           
gastoMonetario:productosAlimenticios11                2719.75                    44.53                   36.25                   19
gastoMonetario:bebidasNoAlcoholicas12                  517.56                    60.23                   71.13                   16
gastoMonetario:bebidasAlcoholicas21                    486.43                    60.93                   77.66                   11
gastoMonetario:tabaco22                                380.09                   114.50                  173.08                   12
gastoMonetario:articulosDeVestir31                    2250.24                    60.15                   81.37                   12
gastoMonetario:calzado32                               841.36               

In [59]:
df_relativos_sorted

,mean_relative_error_pct,std_relative_error_pct,num_hogares_nonzero,mae_eur
variable,,,,
gastoMonetario:productosAlimenticios11,44.525524,36.247198,19,2719.747199
gastoMonetario:bebidasNoAlcoholicas12,60.230982,71.125431,16,517.557302
gastoMonetario:bebidasAlcoholicas21,60.931619,77.659420,11,486.427640
gastoMonetario:tabaco22,114.496674,173.075062,12,380.092170
gastoMonetario:articulosDeVestir31,60.153157,81.373620,12,2250.242866
gastoMonetario:calzado32,87.132487,86.660027,11,841.360624
gastoMonetario:alquileresRealesDeLaVivienda41,29.129971,26.677260,9,1405.589062
gastoMonetario:mantenimientoDeLaVivienda43,136.405046,130.468479,5,538.008782
gastoMonetario:suministroDeAgua44,52.739055,42.082574,20,386.793765


In [60]:
df_relativos_sorted.to_csv("error_relativos.tsv", sep="\t", index=True)

Comprobación

In [68]:
import requests
import json
from time import sleep

def inferir_gastos_hogares(df, columnas_booleanas, ccaa, api_id, delay=0):
    """
    Envía los datos de cada hogar a la API de inferencia para predecir sus gastos monetarios.
    Retorna un DataFrame con una fila por hogar y columnas de predicción de gasto por categoría.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con los datos de los hogares a predecir (una fila por hogar).
    columnas_booleanas : list of str
        Lista de nombres de columnas que deben tratarse como booleanas al enviar a la API.
    ccaa : str
        Código de la comunidad autónoma (dos dígitos como cadena, e.g. "01").
    api_id : str
        ID de la API del digital twin a la que se hacen las peticiones.
    delay : int, optional
        Tiempo de espera (segundos) entre peticiones para no sobrecargar la API.

    Returns
    -------
    pd.DataFrame
        DataFrame con predicciones de gastos. Índice: `hogar` (ID del hogar), 
        Columnas: variables de gasto monetario, más (opcionalmente) columnas auxiliares.
    """
    url = f"https://picota.io/api/1.0.0/digital-twin/{api_id}/subject/hogar{ccaa}/inference"
    lista_preds = []  # Para acumular predicciones por hogar
    
    # Determinar columnas a enviar (omitimos la columna factor y otras no relevantes para la predicción)
    cols_enviar = [col for col in df.columns if col != "factor"]
    
    for i, row in df[cols_enviar].iterrows():
        payload = row.to_dict()
        # Convertir columnas booleanas a tipo bool en el payload
        for col in columnas_booleanas:
            if col in payload:
                valor = payload[col]
                # Si viene como string ("True"/"False"), convertir 
                payload[col] = (str(valor).lower() == "true") if isinstance(valor, str) else bool(valor)
        # Renombrar campo de tiempo para la API
        if "timestamp" in payload:
            payload["instant"] = payload.pop("timestamp")
        else:
            # Si no hay timestamp, igualmente definimos "instant" con algún valor temporal único
            payload["instant"] = f"hogar{ccaa}-row{i}"
        # Enviar petición
        try:
            response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(payload))
        except Exception as e:
            print(f"⚠️ Error de conexión al inferir hogar {i} (CCAA {ccaa}): {e}")
            continue
        # Controlar respuesta de la API
        if response.status_code != 200:
            print(f"⚠️ Respuesta {response.status_code} inferiendo hogar {i} (CCAA {ccaa}): {response.text}")
            continue
        try:
            pred = response.json()
        except json.JSONDecodeError:
            print(f"⚠️ Respuesta no es JSON válido para hogar {i}: {response.text}")
            continue
        if not pred:
            # Si la API devuelve un cuerpo vacío o nulo
            print(f"⚠️ Sin predicciones para hogar {i} (CCAA {ccaa}).")
            continue
        # Convertir la lista de predicciones en DataFrame
        df_pred = pd.DataFrame(pred)
        # Añadir identificador del hogar para poder pivotar
        df_pred["hogar"] = i
        lista_preds.append(df_pred)
        # Esperar un momento antes de la siguiente petición (si se especificó delay)
        if delay:
            sleep(delay)
    
    if not lista_preds:
        return pd.DataFrame()  # Si ninguna predicción fue exitosa
    
    # Concatenar todas las predicciones y pivotar para obtener una fila por hogar
    df_preds_total = pd.concat(lista_preds, ignore_index=True)
    df_wide = df_preds_total.pivot_table(index="hogar", columns="variable", values="value")
    # Asegurar que el índice "hogar" está en orden ascendente (coincide con df original) 
    # y sin índices faltantes (por si algún hogar no tuvo predicción, rellenar con NaN)
    df_wide = df_wide.reindex(range(len(df)), fill_value=np.nan)
    # Añadir columna de subject (identificador del dataset/hogar) para referencias futuras
    df_wide["subject"] = f"hogar{ccaa}"
    return df_wide


In [69]:
bool_cols = [
    'capitalProvincia', 'aguaCaliente', 'calefaccion', 
    'espanolSp', 'educacionSuperiorSp'
]

In [70]:
df = leer_tsv_ccaa(ccaa, ruta_tsvs)

✅ Leyendo archivo: hogar09.tsv


In [71]:
API_ID = "b9ed413b-68c7-487f-bea8-81887a538521"

In [76]:
import pandas as pd
import numpy as np

# 1. Tomamos un hogar de referencia
hogar_base = df.iloc[[7]]  # Cambia por el índice deseado

# 2. Definimos escenarios de ingreso
multiplicadores = [1.0, 1.25, 2.0]  # Desde sin cambio hasta el doble
columna_ingresos = "ingresosNetos"  # Ajusta al nombre exacto de tu variable de ingresos

# 3. Preparamos los distintos hogares modificados
hogares_modificados = pd.concat([
    hogar_base.assign(**{columna_ingresos: hogar_base[columna_ingresos].values[0] * m})
    for m in multiplicadores
], ignore_index=True)

# 4. Inferimos los gastos
preds = inferir_gastos_hogares(hogares_modificados, bool_cols, ccaa="01", api_id=API_ID, delay=1)

# 5. Sumamos el gasto total por hogar
gasto_total = preds.drop(columns=["subject"], errors="ignore").sum(axis=1)
resultados = pd.DataFrame({
    "multiplicador_ingresos": multiplicadores,
    "gasto_total_estimado": gasto_total
})

# 6. Calculamos variación relativa
resultados["variacion_%"] = 100 * (resultados["gasto_total_estimado"] - resultados["gasto_total_estimado"].iloc[0]) / resultados["gasto_total_estimado"].iloc[0]



In [75]:
resultados

,multiplicador_ingresos,gasto_total_estimado,variacion_%
hogar,,,
0,1.00,26967.731623,0.000000
1,1.10,29489.275116,9.350225
2,1.25,33483.044650,24.159663
3,1.50,40384.210994,49.750122
4,2.00,52537.547364,94.816339
